## Tarea algoritmo genético

In [3]:
import random
import numpy as np

# List of movies with attributes: title, genre, and size in GB.
movies = [
    {"title": "The Last House on the Left", "genre": "HORROR", "size": 1.830},
    {"title": "Saw IV", "genre": "HORROR", "size": 1.435},
    {"title": "Orphan", "genre": "HORROR", "size": 2.163},
    {"title": "Clash of the Titans", "genre": "ACTION", "size": 1.746},
    {"title": "Man of Steel", "genre": "ACTION", "size": 0.964},
    {"title": "The Avengers", "genre": "ACTION", "size": 2.032},
    {"title": "American Reunion", "genre": "COMEDY", "size": 1.257},
    {"title": "Silver Linings Playbook", "genre": "COMEDY", "size": 3.139},
    {"title": "The Three Stooges", "genre": "COMEDY", "size": 0.750},
    {"title": "Wild Card", "genre": "THRILLER", "size": 2.275},
    {"title": "The Body", "genre": "THRILLER", "size": 2.082},
    {"title": "15 Years and One Day", "genre": "THRILLER", "size": 2.321}
]

# Restrictions: certain movies cannot be selected together, represented as pairs.
movie_restrictions = [
    ("Wild Card", "The Body"),
    ("Clash of the Titans", "Man of Steel")
]

# Genre restrictions: restrict selecting both genres in a single DVD.
genre_restrictions = ("COMEDY", "HORROR")

# Available DVD sizes in GB.
dvd_sizes = [4.7, 8.5, 13.3, 15.9]

# Genetic algorithm parameters.
NUM_INDIVIDUALS = 100  # Population size
GENERATIONS = 100      # Number of generations
MUTATION_PROBABILITY = 0.1  # Probability of mutation

def calculate_fitness(individual, dvd_size):
    # Total size of selected movies.
    total_size = sum(m["size"] for i, m in enumerate(movies) if individual[i] == 1)

    # If total size exceeds DVD capacity, return a fitness score of 0.
    if total_size > dvd_size:
        return 0

    # Check genre restrictions: no selection should include both restricted genres.
    genres = {movies[i]["genre"] for i in range(len(movies)) if individual[i] == 1}
    if genre_restrictions[0] in genres and genre_restrictions[1] in genres:
        return 0

    # Check pair restrictions: restricted pairs should not be selected together.
    for restriction in movie_restrictions:
        if any(m["title"] == restriction[0] for i, m in enumerate(movies) if individual[i] == 1) and \
           any(m["title"] == restriction[1] for i, m in enumerate(movies) if individual[i] == 1):
            return 0

    return total_size  # Return the total size if all restrictions are met.

def create_individual():
    # Create a random individual (a random binary selection of movies).
    return [random.randint(0, 1) for _ in movies]

def selection(population, dvd_size):
    # Calculate fitness for each individual and sort by fitness.
    scores = [(ind, calculate_fitness(ind, dvd_size)) for ind in population]
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    # Select top 50% individuals for mating.
    selected = [s[0] for s in scores[:NUM_INDIVIDUALS // 2]]
    return selected

def crossover(ind1, ind2):
    # Single-point crossover between two parents.
    point = random.randint(1, len(ind1) - 2)
    return ind1[:point] + ind2[point:], ind2[:point] + ind1[point:]

def mutation(individual):
    # Apply mutation with a given probability.
    if random.random() < MUTATION_PROBABILITY:
        point = random.randint(0, len(individual) - 1)
        individual[point] = 1 - individual[point]  # Flip the binary bit at the mutation point.

def genetic_algorithm(dvd_size):
    # Initialize population with random individuals.
    population = [create_individual() for _ in range(NUM_INDIVIDUALS)]

    for _ in range(GENERATIONS):
        # Selection of the top individuals for reproduction.
        selected = selection(population, dvd_size)
        new_population = []
        # Generate new individuals by crossover and mutation.
        while len(new_population) < NUM_INDIVIDUALS:
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            child1, child2 = crossover(parent1, parent2)
            mutation(child1)
            mutation(child2)
            new_population.extend([child1, child2])
        population = new_population  # Update population with the new generation.

    # Find the best individual with the highest fitness value.
    best_individual = max(population, key=lambda ind: calculate_fitness(ind, dvd_size))
    best_fitness = calculate_fitness(best_individual, dvd_size)

    # Print the best solution for the given DVD size.
    selected_titles = [movies[i]["title"] for i in range(len(movies)) if best_individual[i] == 1]
    print(f"Best fit for DVD of {dvd_size} GB:")
    print(f"Selected movies: {selected_titles}")
    print(f"Used space: {best_fitness:.3f} GB\n")

# Run the genetic algorithm for each DVD size.
for size in dvd_sizes:
    genetic_algorithm(size)


Best fit for DVD of 4.7 GB:
Selected movies: ['Wild Card', '15 Years and One Day']
Used space: 4.596 GB

Best fit for DVD of 8.5 GB:
Selected movies: ['Clash of the Titans', 'American Reunion', 'Silver Linings Playbook', '15 Years and One Day']
Used space: 8.463 GB

Best fit for DVD of 13.3 GB:
Selected movies: ['Man of Steel', 'The Avengers', 'American Reunion', 'Silver Linings Playbook', 'The Three Stooges', 'Wild Card', '15 Years and One Day']
Used space: 12.738 GB

Best fit for DVD of 15.9 GB:
Selected movies: ['Clash of the Titans', 'The Avengers', 'American Reunion', 'Silver Linings Playbook', 'The Three Stooges', 'Wild Card', '15 Years and One Day']
Used space: 13.520 GB

